In [1]:
!pip install tqdm

    100% |████████████████████████████████| 51kB 2.6MB/s 


In [2]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [4]:
import zipfile
from tqdm import trange,tqdm
dset = zipfile.ZipFile('drive/Data/UTKFace.zip')
ilist = dset.infolist()
for i in trange(len(ilist)):
    dset.extract(ilist[i])

100%|██████████| 23708/23708 [00:06<00:00, 3672.75it/s]


# Race Prediction on LFW

## Import Libraries

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential,load_model
from keras.layers import Convolution2D,MaxPooling2D,Dense,Dropout,Activation,Flatten
from keras.callbacks import ModelCheckpoint,EarlyStopping,TensorBoard
from keras.utils import to_categorical
import seaborn as sns
from sklearn.metrics import accuracy_score,classification_report
from sklearn.ensemble import RandomForestRegressor
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import class_weight
from sklearn import svm
import random
import h5py
import cv2
from glob import glob
from tqdm import tqdm,trange
import re

Using TensorFlow backend.


In [0]:
num_epochs=50
batch_size = 64

## Load Data

In [0]:
filelist = glob("UTKFace/*.jpg")
labels = [re.search(r'UTKFace/(\d+)_(\d+)_(\d+).*',i).groups() for i in filelist]
labels = [[int(j) for j in i] for i in labels]
labels = np.array(labels)

In [0]:
def load_image(filepath):
    img = cv2.imread(filepath)
    img = img[25:-25,25:-25,:]
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img = cv2.resize(img,(64,64))
    return img.astype(np.float32)/255.0

In [0]:
X = np.array([load_image(i) for i in filelist])

In [0]:
y = labels[:,0]

## VGG-16 Model

In [11]:
model = Sequential()
model.add(Convolution2D(32,(3,3),activation='relu',input_shape=(64,64,3),padding='same'))
model.add(Convolution2D(32,(3,3),activation='relu',padding='same'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Convolution2D(64,(3,3),activation='relu',padding='same'))
model.add(Convolution2D(64,(3,3),activation='relu',padding='same'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Convolution2D(128,(3,3),activation='relu',padding='same'))
model.add(Convolution2D(128,(3,3),activation='relu',padding='same'))
model.add(Flatten())
model.add(Dense(1024,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1024,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam',metrics=['mean_absolute_error'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 64, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 16, 16, 128)       73856     
__________

In [0]:
es_callback = EarlyStopping(patience=3,verbose=1)
callback_list = [es_callback]

## Training the Model

In [0]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [13]:
model.fit(X_train,y_train,epochs=10,verbose=1,validation_data=(X_test,y_test))

Train on 18966 samples, validate on 4742 samples
Epoch 1/10
18966/18966 [==============================] - 60s 3ms/step - loss: 403.0377 - mean_absolute_error: 15.1462 - val_loss: 231.4609 - val_mean_absolute_error: 11.1685
Epoch 2/10
  288/18966 [..............................] - ETA: 54s - loss: 246.6186 - mean_absolute_error: 12.1088

18966/18966 [==============================] - 57s 3ms/step - loss: 185.8211 - mean_absolute_error: 10.1220 - val_loss: 125.9337 - val_mean_absolute_error: 8.2573
Epoch 3/10
 4608/18966 [======>.......................] - ETA: 41s - loss: 143.6008 - mean_absolute_error: 8.8647

18966/18966 [==============================] - 57s 3ms/step - loss: 136.2538 - mean_absolute_error: 8.5538 - val_loss: 110.5507 - val_mean_absolute_error: 7.5796
Epoch 4/10
 6272/18966 [========>.....................] - ETA: 36s - loss: 116.0910 - mean_absolute_error: 7.9617

18966/18966 [==============================] - 57s 3ms/step - loss: 117.5000 - mean_absolute_error: 7.9595 - val_loss: 91.5905 - val_mean_absolute_error: 6.9843
Epoch 5/10
 6912/18966 [=========>....................] - ETA: 34s - loss: 107.4973 - mean_absolute_error: 7.6000

18966/18966 [==============================] - 57s 3ms/step - loss: 105.4966 - mean_absolute_error: 7.5341 - val_loss: 85.8636 - val_mean_absolute_error: 6.7013
Epoch 6/10
 7200/18966 [==========>...................] - ETA: 33s - loss: 91.4405 - mean_absolute_error: 6.9648

18966/18966 [==============================] - 57s 3ms/step - loss: 92.6483 - mean_absolute_error: 7.0361 - val_loss: 84.7358 - val_mean_absolute_error: 6.7557
Epoch 7/10
 7360/18966 [==========>...................] - ETA: 33s - loss: 80.4670 - mean_absolute_error: 6.5503

18966/18966 [==============================] - 57s 3ms/step - loss: 82.0182 - mean_absolute_error: 6.6230 - val_loss: 85.6982 - val_mean_absolute_error: 6.7618
Epoch 8/10
 7424/18966 [==========>...................] - ETA: 33s - loss: 75.1641 - mean_absolute_error: 6.3367

18966/18966 [==============================] - 57s 3ms/step - loss: 76.7085 - mean_absolute_error: 6.3783 - val_loss: 90.3517 - val_mean_absolute_error: 6.7235
Epoch 9/10
 7456/18966 [==========>...................] - ETA: 33s - loss: 67.8143 - mean_absolute_error: 6.0274

18966/18966 [==============================] - 57s 3ms/step - loss: 67.1225 - mean_absolute_error: 5.9792 - val_loss: 86.3814 - val_mean_absolute_error: 6.5764
Epoch 10/10
 7424/18966 [==========>...................] - ETA: 33s - loss: 59.2978 - mean_absolute_error: 5.6701

18966/18966 [==============================] - 57s 3ms/step - loss: 58.8909 - mean_absolute_error: 5.6236 - val_loss: 82.8925 - val_mean_absolute_error: 6.4157


In [0]:
model.load_weights("../Models/LFW_GRAYSCALE_64/Race/no-dropout-001-0.7030.hdf5")
y_pred = model.predict(X_train)

In [0]:
y_pred_int = []
for i in range(len(y_pred)):
    for j in range(len(y_pred[i])):
        if y_pred[i][j]==max(y_pred[i]):
            y_pred_int.append(j)
            break

## Results

In [0]:
print(accuracy_score(y_train,y_pred_int))

0.81039336529


In [0]:
import os
os.path.exists('lfw-deepfunneled\\Aaron_Eckhart')

False

In [0]:
!ls lfw-deepfunneled/Aaron_Eckhart

Aaron_Eckhart_0001.jpg


In [0]:
type(model)

keras.models.Sequential

In [0]:
y_pred = model.predict(X_test)

In [0]:
def onehot_to_cat(y_pred):
  y_pred_int = []
  for i in y_pred:
    mi = max(i)
    for j in range(8):
      if(i[j]==mi):
        y_pred_int.append(j)
        break
  return np.array(y_pred_int)

In [0]:
y_pred_int = onehot_to_cat(y_pred)
y_true_int = onehot_to_cat(y_test)

In [0]:
print(classification_report(y_true_int,y_pred_int))

             precision    recall  f1-score   support

          0       0.82      0.84      0.83      2003
          1       0.83      0.81      0.82       896
          2       0.80      0.85      0.82       689
          3       0.69      0.73      0.71       818
          4       0.35      0.25      0.29       334
          6       0.00      0.00      0.00         1
          7       0.00      0.00      0.00         1

avg / total       0.76      0.77      0.77      4742



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [0]:
del lfwdf

In [0]:
print(re.search(r'UTKFace/(\d+)_(\d+)_(\d+).*',filelist[0]).groups())

('26', '1', '3')


In [0]:
filelist[0]

'UTKFace/26_1_3_20170117153635595.jpg.chip.jpg'

In [0]:
np.unique(labels[:,2],return_counts=True)

(array([                0,                 1,                 2,
                        3,                 4, 20170109142408075,
        20170109150557335, 20170116174525125]),
 array([10078,  4526,  3434,  3975,  1692,     1,     1,     1]))

In [0]:
X

array([[26,  1,  3],
       [68,  0,  0],
       [26,  1,  0],
       ...,
       [28,  0,  4],
       [90,  1,  1],
       [32,  1,  2]])

In [0]:
model.save("drive/Models/LFW_RGB_64/Age/AgeNet.hdf5")

In [0]:
!mkdir drive/Models/LFW_RGB_64/Age